In [1]:
#importing, install libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [2]:
address = 'Manila, Philippines'

geolocator = Nominatim(user_agent="ph_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Metro Manila are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Metro Manila are 14.5906216, 120.9799696.


In [3]:
# create map of Manila using latitude and longitude values
map_metromanila = folium.Map(location=[latitude, longitude], zoom_start=10)
map_metromanila

In [4]:
CLIENT_ID = 'V253AN4OKKK034FDX0GI0YGBLP0MQTA3NYZC3QIWLRC0NOMV' # your Foursquare ID
CLIENT_SECRET = 'PJBZTIK5XIH2OJDBCMCEG1Q55YG0SNZE05WZVHGHL3MWLUUX' # your Foursquare Secret
VERSION = '20200611' # Foursquare API version 20180605

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V253AN4OKKK034FDX0GI0YGBLP0MQTA3NYZC3QIWLRC0NOMV
CLIENT_SECRET:PJBZTIK5XIH2OJDBCMCEG1Q55YG0SNZE05WZVHGHL3MWLUUX


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
cities_data = pd.read_csv("Cities_Towns_PH_Coordinates.csv")
cities_df = pd.DataFrame(cities_data, columns = ['Place Name', 'Latitude', 'Longitude'])
cities_df.head()

,Place Name,Latitude,Longitude
0,"Maasin City, Southern Leyte, Philippinnes",10.13140,124.85848
1,"Cagayan de Oro, Misamis Oriental, Philippines",8.47722,124.64592
2,"Silay City, Negros Occidental, Philippines",10.75379,123.08416
3,"Taytay, Rizal, Philippines",14.55856,121.13609
4,"Bago City, Negros Occidental, Philippines",10.50341,122.96630


In [7]:
cities_df.rename(columns={'Place Name': 'Neighborhood'}, inplace=True)
cities_df.tail(10)

,Neighborhood,Latitude,Longitude
110,"Paete, Laguna, Philippines",14.38015,121.52321
111,"Caloocan City, Metro Manila, Philippines",14.64953,120.96788
112,"Las Pinas City, Metro Manila, Philippines",14.45056,120.98278
113,"Malabon City, Metro Manila, Philippines",14.66810,120.96580
114,"Mandaluyong City, Metro Manila, Philippines",14.58320,121.04090
115,"Marikina City, Metro Manila, Philippines",14.65070,121.10290
116,"Navotas City, Metro Manila, Philippines",14.66667,120.95000
117,"Paranaque City, Metro Manila, Philippines",14.47930,121.01980
118,"Pasig City, Metro Manila, Philippines",14.58691,121.06140
119,"San Juan City, Metro Manila, Philippines",14.60000,121.03302


In [8]:
#extract metro manila cities only
manila_data = cities_df[cities_df['Neighborhood'].str.contains('Metro Manila')]
manila_data.reset_index(inplace=True, drop=True)
manila_data.head(20)

,Neighborhood,Latitude,Longitude
0,"Pasay, Metro Manila, Philippines",14.53775,121.00138
1,"Muntinlupa, Metro Manila, Philippines",14.40813,121.04147
2,"Valenzuela, Metro Manila, Philippines",14.70358,120.98654
3,"Manila, Metro Manila, Philippines",14.59951,120.98422
4,"Makati, Metro Manila, Philippines",14.55659,121.02342
5,"Taguig City, Metro Manila, Philippines",14.52045,121.05389
6,"Quezon City, Metro Manila, Philippines",14.67621,121.04386
7,"Caloocan City, Metro Manila, Philippines",14.64953,120.96788
8,"Las Pinas City, Metro Manila, Philippines",14.45056,120.98278
9,"Malabon City, Metro Manila, Philippines",14.66810,120.96580


In [9]:
manila_data['Neighborhood'] = manila_data['Neighborhood'].str.replace(', Metro Manila, Philippines','')
#manila_data['Neighborhood'].loc[manila_data['Neighborhood'] == 'Metro Manila, Philippines'].replace("", inplace=True, regex=True)
manila_data.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Latitude,Longitude
0,Pasay,14.53775,121.00138
1,Muntinlupa,14.40813,121.04147
2,Valenzuela,14.70358,120.98654
3,Manila,14.59951,120.98422
4,Makati,14.55659,121.02342
5,Taguig City,14.52045,121.05389
6,Quezon City,14.67621,121.04386
7,Caloocan City,14.64953,120.96788
8,Las Pinas City,14.45056,120.98278
9,Malabon City,14.66810,120.96580


In [10]:
manila_venues = getNearbyVenues(names=manila_data['Neighborhood'],
                                   latitudes=manila_data['Latitude'],
                                   longitudes=manila_data['Longitude']
                                  )

Pasay
Muntinlupa
Valenzuela
Manila
Makati
Taguig City
Quezon City
Caloocan City
Las Pinas City
Malabon City
Mandaluyong City
Marikina City
Navotas City
Paranaque City
Pasig City
San Juan City


In [14]:
print(manila_venues.shape)
manila_venues.head(10)

(1441, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pasay,14.53775,121.00138,The Henry Manila,14.541000,120.995939,Hotel
1,Pasay,14.53775,121.00138,Avellana Art Gallery,14.540902,120.996322,Art Gallery
2,Pasay,14.53775,121.00138,S&R Food Counter,14.529792,120.991295,Fast Food Restaurant
3,Pasay,14.53775,121.00138,Conrad's Grille,14.526522,120.999771,BBQ Joint
4,Pasay,14.53775,121.00138,Starbucks,14.537152,120.989173,Coffee Shop
5,Pasay,14.53775,121.00138,The Wine Museum,14.535033,121.003397,Wine Bar
6,Pasay,14.53775,121.00138,S&R Membership Shopping,14.529834,120.991173,Supermarket
7,Pasay,14.53775,121.00138,Tim Hortons,14.536695,120.990655,Coffee Shop
8,Pasay,14.53775,121.00138,Velvet Friends Cat Cafe,14.537952,120.989788,Pet Café
9,Pasay,14.53775,121.00138,Starbucks,14.536488,120.988724,Coffee Shop


In [15]:
manila_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Caloocan City,94,94,94,94,94,94
Las Pinas City,98,98,98,98,98,98
Makati,100,100,100,100,100,100
Malabon City,93,93,93,93,93,93
Mandaluyong City,100,100,100,100,100,100
Manila,100,100,100,100,100,100
Marikina City,80,80,80,80,80,80
Muntinlupa,100,100,100,100,100,100
Navotas City,59,59,59,59,59,59


In [33]:
#Categories = manila_venues['Venue Category'].unique()
#print('There are {} uniques categories.'.format(len(manila_venues['Venue Category'].unique())))
category_food = manila_venues[manila_venues['Venue Category'].str.contains('Restaurant' or 'Food')]
category_food.reset_index(inplace=True, drop=True)
category_food.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pasay,14.53775,121.00138,S&R Food Counter,14.529792,120.991295,Fast Food Restaurant
1,Pasay,14.53775,121.00138,Kaito Japanese Carinderia,14.543683,121.012782,Japanese Restaurant
2,Pasay,14.53775,121.00138,Conti's Bakeshop & Restaurant,14.537241,120.989985,Restaurant
3,Pasay,14.53775,121.00138,Spare Strike,14.540995,121.011987,Tapas Restaurant
4,Pasay,14.53775,121.00138,The Tasting Room,14.524791,120.993227,French Restaurant


In [37]:
map_foods = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
colors = 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(category_food['Venue Latitude'], category_food['Venue Longitude'], category_food['Neighborhood'], category_food['Venue']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_foods)
       
map_foods

In [17]:
# one hot encoding
manila_onehot = pd.get_dummies(manila_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manila_onehot['Neighborhood'] = manila_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manila_onehot.columns[-1]] + list(manila_onehot.columns[:-1])
manila_onehot = manila_onehot[fixed_columns]

manila_onehot.head()

,Zoo,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Garden,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Casino,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Quad,College Science Building,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Doctor's Office,Donut Shop,Dumpling Restaurant,Empanada Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Movie Theater,Multiplex,Nail Salon,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paella Restaurant,Park,Pet Café,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Radio Station,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Park,Tonkatsu Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pasay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pasay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pasay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pasay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
manila_onehot.shape

(1441, 183)

In [19]:
manila_grouped = manila_onehot.groupby('Neighborhood').mean().reset_index()
manila_grouped

,Neighborhood,Zoo,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Garden,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Casino,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Quad,College Science Building,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Doctor's Office,Donut Shop,Dumpling Restaurant,Empanada Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Movie Theater,Multiplex,Nail Salon,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paella Restaurant,Park,Pet Café,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Radio Station,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Park,Tonkatsu Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Caloocan City,0.000000,0.010638,0.00,0.010638,0.0000,0.00,0.00,0.031915,0.000000,0.00,0.000000,0.031915,0.000000,0.010638,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.021277,0.000000,0.00,0.000000,0.010638,0.010638,0.00,0.000000,0.010638,0.000000,0.053191,0.00,0.000000,0.117021,0.000000,0.000000,0.00,0.00,0.010638,0.00,0.00,0.00,0.0000,0.000000,0.000000,0.00,0.106383,0.000000,0.00,0.010638,0.00,0.000000,0.00,0.000000,0.000000,0.021277,0.00,0.00,0.031915,0.00,0.00,0.00,0.244681,0.031915,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010638,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.010638,0.000000,0.00,0.042553,0.0000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.021277,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.010638,0.00,0.00,0.00,0.010638,0.000000,0.00,0.00,0.000000,0.00,0.00,0.010638,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.021277,0.00,0.000000,0.000000,0.010638,0.00,0.00,0.0000,0.00,0.0000,0.00,0.000000,0.010638,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000,0.010638,0.00,0.000000,0.00,0.000000,0.00,0.010638,0.00,0.000000,0.00,0.0000,0.000000,0.021277,0.00,0.00,0.00,0.010638,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
1,Las Pinas City,0.000000,0.010204,0.00,0.000000,0.0000,0.00,0.00,0.010204,0.

In [20]:
num_top_venues = 5

for hood in manila_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manila_grouped[manila_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caloocan City----
                  venue  freq
0  Fast Food Restaurant  0.24
1    Chinese Restaurant  0.12
2     Convenience Store  0.11
3                  Café  0.05
4   Japanese Restaurant  0.04


----Las Pinas City----
                  venue  freq
0  Fast Food Restaurant  0.09
1     Convenience Store  0.07
2   Japanese Restaurant  0.06
3   Filipino Restaurant  0.05
4                   Spa  0.05


----Makati----
                 venue  freq
0          Coffee Shop  0.11
1                Hotel  0.07
2        Shopping Mall  0.07
3  Japanese Restaurant  0.05
4                 Café  0.04


----Malabon City----
                  venue  freq
0  Fast Food Restaurant  0.17
1     Convenience Store  0.10
2                  Café  0.08
3    Chinese Restaurant  0.05
4          Burger Joint  0.04


----Mandaluyong City----
                 venue  freq
0  Filipino Restaurant  0.10
1  Japanese Restaurant  0.09
2   Chinese Restaurant  0.06
3          Coffee Shop  0.06
4        Shopping Mall  0.0

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manila_grouped['Neighborhood']

for ind in np.arange(manila_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manila_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caloocan City,Fast Food Restaurant,Chinese Restaurant,Convenience Store,Café,Japanese Restaurant,BBQ Joint,Filipino Restaurant,Donut Shop,Asian Restaurant,Bookstore
1,Las Pinas City,Fast Food Restaurant,Convenience Store,Japanese Restaurant,Spa,Filipino Restaurant,Bar,Bubble Tea Shop,Bakery,Tea Room,Café
2,Makati,Coffee Shop,Shopping Mall,Hotel,Japanese Restaurant,Café,Burger Joint,Spanish Restaurant,Clothing Store,Yoga Studio,Bakery
3,Malabon City,Fast Food Restaurant,Convenience Store,Café,Chinese Restaurant,Bubble Tea Shop,Filipino Restaurant,Burger Joint,Asian Restaurant,Tea Room,Donut Shop
4,Mandaluyong City,Filipino Restaurant,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Shopping Mall,Spa,Bakery,Italian Restaurant,Buffet,Supermarket


In [23]:
# set number of clusters
kclusters = 8

manila_grouped_clustering = manila_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manila_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 2, 1, 2, 4, 5, 7, 1, 5], dtype=int32)

In [24]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manila_merged = manila_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manila_merged = manila_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manila_merged.head(10) # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pasay,14.53775,121.00138,0,Coffee Shop,Seafood Restaurant,Chinese Restaurant,Hotel,Korean Restaurant,Burger Joint,Restaurant,Japanese Restaurant,Asian Restaurant,BBQ Joint
1,Muntinlupa,14.40813,121.04147,7,Fast Food Restaurant,Coffee Shop,Filipino Restaurant,Hotel,Pizza Place,Asian Restaurant,Donut Shop,Yoga Studio,Supermarket,Hotel Bar
2,Valenzuela,14.70358,120.98654,6,Convenience Store,Fast Food Restaurant,Pizza Place,Pool,Food Court,Spa,Coffee Shop,Supermarket,Casino,Rest Area
3,Manila,14.59951,120.98422,4,Chinese Restaurant,Filipino Restaurant,Bakery,Bubble Tea Shop,Coffee Shop,Café,Pizza Place,Japanese Restaurant,Hotel,Tea Room
4,Makati,14.55659,121.02342,2,Coffee Shop,Shopping Mall,Hotel,Japanese Restaurant,Café,Burger Joint,Spanish Restaurant,Clothing Store,Yoga Studio,Bakery
5,Taguig City,14.52045,121.05389,7,Fast Food Restaurant,Coffee Shop,Café,Restaurant,Chinese Restaurant,Pizza Place,Convenience Store,Sporting Goods Shop,Asian Restaurant,Shopping Mall
6,Quezon City,14.67621,121.04386,3,Pizza Place,Convenience Store,Fast Food Restaurant,Bakery,Filipino Restaurant,BBQ Joint,Burger Joint,Spa,Chinese Restaurant,Bar
7,Caloocan City,14.64953,120.96788,1,Fast Food Restaurant,Chinese Restaurant,Convenience Store,Café,Japanese Restaurant,BBQ Joint,Filipino Restaurant,Donut Shop,Asian Restaurant,Bookstore
8,Las Pinas City,14.45056,120.98278,3,Fast Food Restaurant,Convenience Store,Japanese Restaurant,Spa,Filipino Restaurant,Bar,Bubble Tea Shop,Bakery,Tea Room,Café
9,Malabon City,14.66810,120.96580,1,Fast Food Restaurant,Convenience Store,Café,Chinese Restaurant,Bubble Tea Shop,Filipino Restaurant,Burger Joint,Asian Restaurant,Tea Room,Donut Shop


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manila_merged['Latitude'], manila_merged['Longitude'], manila_merged['Neighborhood'], manila_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
manila_merged.loc[manila_merged['Cluster Labels'] == 0]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pasay,14.53775,121.00138,0,Coffee Shop,Seafood Restaurant,Chinese Restaurant,Hotel,Korean Restaurant,Burger Joint,Restaurant,Japanese Restaurant,Asian Restaurant,BBQ Joint


In [28]:
manila_merged.loc[manila_merged['Cluster Labels'] == 1]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Caloocan City,14.64953,120.96788,1,Fast Food Restaurant,Chinese Restaurant,Convenience Store,Café,Japanese Restaurant,BBQ Joint,Filipino Restaurant,Donut Shop,Asian Restaurant,Bookstore
9,Malabon City,14.66810,120.96580,1,Fast Food Restaurant,Convenience Store,Café,Chinese Restaurant,Bubble Tea Shop,Filipino Restaurant,Burger Joint,Asian Restaurant,Tea Room,Donut Shop
12,Navotas City,14.66667,120.95000,1,Fast Food Restaurant,Filipino Restaurant,Chinese Restaurant,Convenience Store,Tea Room,Bubble Tea Shop,Donut Shop,Shopping Mall,Spa,Steakhouse


In [93]:
manila_merged.loc[manila_merged['Cluster Labels'] == 2]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Makati,14.55659,121.02342,2,Coffee Shop,Shopping Mall,Hotel,Japanese Restaurant,Café,Burger Joint,Spanish Restaurant,Clothing Store,Yoga Studio,Bakery
10,Mandaluyong City,14.58320,121.04090,2,Filipino Restaurant,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Shopping Mall,Spa,Bakery,Italian Restaurant,Buffet,Supermarket
14,Pasig City,14.58691,121.06140,2,Coffee Shop,Shopping Mall,Café,Clothing Store,Filipino Restaurant,Buffet,Japanese Restaurant,Bakery,Pizza Place,Italian Restaurant


In [94]:
manila_merged.loc[manila_merged['Cluster Labels'] == 3]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Quezon City,14.67621,121.04386,3,Pizza Place,Convenience Store,Fast Food Restaurant,Bakery,Filipino Restaurant,BBQ Joint,Burger Joint,Spa,Chinese Restaurant,Bar
8,Las Pinas City,14.45056,120.98278,3,Fast Food Restaurant,Convenience Store,Japanese Restaurant,Spa,Filipino Restaurant,Bar,Bubble Tea Shop,Bakery,Tea Room,Café


In [95]:
manila_merged.loc[manila_merged['Cluster Labels'] == 4]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Manila,14.59951,120.98422,4,Chinese Restaurant,Filipino Restaurant,Bakery,Bubble Tea Shop,Coffee Shop,Café,Pizza Place,Japanese Restaurant,Hotel,Tea Room
15,San Juan City,14.60000,121.03302,4,Chinese Restaurant,Filipino Restaurant,Japanese Restaurant,Coffee Shop,Bakery,Café,Bubble Tea Shop,Bar,Italian Restaurant,Shopping Mall


In [96]:
manila_merged.loc[manila_merged['Cluster Labels'] == 5]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Marikina City,14.6507,121.1029,5,Convenience Store,Fast Food Restaurant,Comfort Food Restaurant,Café,Steakhouse,Diner,Burger Joint,Restaurant,Filipino Restaurant,Pizza Place
13,Paranaque City,14.4793,121.0198,5,Fast Food Restaurant,Convenience Store,Café,Grocery Store,Pharmacy,Gym / Fitness Center,Japanese Restaurant,Pizza Place,Gym,Spa


In [65]:
manila_merged.loc[manila_merged['Cluster Labels'] == 6]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Valenzuela,14.70358,120.98654,6,Convenience Store,Fast Food Restaurant,Pizza Place,Pool,Food Court,Spa,Coffee Shop,Supermarket,Casino,Rest Area


In [66]:
manila_merged.loc[manila_merged['Cluster Labels'] == 7]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Muntinlupa,14.40813,121.04147,7,Fast Food Restaurant,Coffee Shop,Filipino Restaurant,Hotel,Pizza Place,Asian Restaurant,Donut Shop,Yoga Studio,Supermarket,Hotel Bar
5,Taguig City,14.52045,121.05389,7,Fast Food Restaurant,Coffee Shop,Café,Restaurant,Chinese Restaurant,Pizza Place,Convenience Store,Sporting Goods Shop,Asian Restaurant,Shopping Mall
